## Naive Bayes Spam Filter Project

The goal of this project is to construct a spam filter based on the naive Bayes algorithm. To achieve this a dataset of over 5000 SMS messages, that have already been categorized as either spam or ham (not spam) will be used to first train and then test the algorithm.

The goal is to reach an accuracy of at least 80%.

# Setup

In [1]:
import numpy as np
import pandas as pd

Transforming the csv file into a pandas dataframe:

In [2]:
spam_col = pd.read_csv('SMSSpamCollection', sep = '\t', header = None, names = ['Label','SMS'])

Check if it worked:

In [3]:
spam_col.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
print(spam_col.shape)

(5572, 2)


The dataset has two columns:
* `Label`: either spam or ham
* `SMS`: text of the message

Finding the percentage of authentic vs spam messages:

In [5]:
print(spam_col.Label.value_counts(normalize = True, dropna = False)*100)

ham     86.593683
spam    13.406317
Name: Label, dtype: float64


A large majority of the messages are ham. This is a problem. A model classifying all messages as ham would have an 86.6 % accuracy, but be absolutely useless in classifying the messages. This needs to be taken into consideration both when training and testing the model.

### Train and test datasets

To ensure a non-biased evaluation and combat overfitting,a separate train and test set will be constructed.

**Overfitting** means that the algorithm is very closely shaped around a single dataset. It "memorizes" each of the datapoints instead of drawing generalizing. Overfitting is very treacherous, because if an overfitted model is tested on the data it was trained on it will perform very strongly. But when such a model is applied to new data, it will perform very poorly. To combat this it is very helpful to keep the train and test sets completely separate.

In this case the dataset will be divided into the two datasets:

* `training_set` containing about 80% of all messages
* `test_set` set with the remaining 20%.

The training set will be used to train our algorithm and the test set will be used to compare the predictions of the algorithm to the manual labeling done by users. We are striving for an accuracy of more than 80%

Creating the two datasets by random sampling:

In [6]:
random_spam = spam_col.sample(frac = 1, random_state = 1).copy()

In [7]:
training_set = random_spam[:4458].copy()
test_set = random_spam[4458:].copy()
print(training_set.shape, test_set.shape)
print(4458/5572)

(4458, 2) (1114, 2)
0.8000717875089735


checking to see if the datasets are well randomized:

In [8]:
print(training_set.Label.value_counts(normalize = True)*100)
print(test_set.Label.value_counts(normalize = True)*100)

ham     86.54105
spam    13.45895
Name: Label, dtype: float64
ham     86.804309
spam    13.195691
Name: Label, dtype: float64


# Data Cleaning:

Here several tasks need to be accomplished:
* make all words lower case, so capitalization does not impact the count
* ignore all punctuation.

This will help in the next step, where the dataset will be transformed for easier analysis.

Initial status of the datasets' heads:

In [9]:
print(training_set.head())
print(test_set.head())

     Label                                                SMS
1078   ham                       Yep, by the pretty sculpture
4028   ham      Yes, princess. Are you going to make me moan?
958    ham                         Welp apparently he retired
4642   ham                                            Havent.
4674   ham  I forgot 2 ask ü all smth.. There's a card on ...
     Label                                                SMS
2131   ham          Later i guess. I needa do mcat study too.
3418   ham             But i haf enuff space got like 4 mb...
3424  spam  Had your mobile 10 mths? Update to latest Oran...
1538   ham  All sounds good. Fingers . Makes it difficult ...
5393   ham  All done, all handed in. Don't know if mega sh...


Each step in data cleaning will be applied to both datasets.

In the first step, all characters that are neither letters, digits or underscores (`\W`) will be replaces with a whitespace.

In [10]:
training_set.SMS = training_set.SMS.str.replace('\W', ' ').str.lower()
test_set.SMS = test_set.SMS.str.replace('\W', ' ').str.lower()

In [11]:
print(training_set.head())
print(test_set.head())

     Label                                                SMS
1078   ham                       yep  by the pretty sculpture
4028   ham      yes  princess  are you going to make me moan 
958    ham                         welp apparently he retired
4642   ham                                            havent 
4674   ham  i forgot 2 ask ü all smth   there s a card on ...
     Label                                                SMS
2131   ham          later i guess  i needa do mcat study too 
3418   ham             but i haf enuff space got like 4 mb   
3424  spam  had your mobile 10 mths  update to latest oran...
1538   ham  all sounds good  fingers   makes it difficult ...
5393   ham  all done  all handed in  don t know if mega sh...


This means all the punctuation has now been removed.

Next, each of the messages will be split using the pandas function `.str.split` at the whitespaces into lists and replace the entries in the `SMS` column.

In [12]:
training_set.SMS = training_set.SMS.str.split()
test_set.SMS = test_set.SMS.str.split()

In [13]:
print(training_set.head())
print(test_set.head())

     Label                                                SMS
1078   ham                  [yep, by, the, pretty, sculpture]
4028   ham  [yes, princess, are, you, going, to, make, me,...
958    ham                    [welp, apparently, he, retired]
4642   ham                                           [havent]
4674   ham  [i, forgot, 2, ask, ü, all, smth, there, s, a,...
     Label                                                SMS
2131   ham  [later, i, guess, i, needa, do, mcat, study, too]
3418   ham      [but, i, haf, enuff, space, got, like, 4, mb]
3424  spam  [had, your, mobile, 10, mths, update, to, late...
1538   ham  [all, sounds, good, fingers, makes, it, diffic...
5393   ham  [all, done, all, handed, in, don, t, know, if,...


The dataset is now formatted to be easier to work with in the next steps!

## Re-formatting the dataset

To make training the model easier, the dataset will now be put into a different form. Instead of using one single column with all the words of a given message, each word will now get its own column containing the count of each word in each message.

First a list of all the words has to be made:

In [14]:
vocabulary_lst = []
for sms in training_set.SMS:
    for word in sms:
        if word not in vocabulary_lst:
            vocabulary_lst.append(word)
    

Ensuring that every word is only present once in the new list

In [15]:
print(pd.Series(vocabulary_lst).value_counts().value_counts())

1    7783
dtype: int64


Reset the indexes in both datasets, since they are still the same as they were before the randomization

In [16]:
training_set.reset_index(inplace = True)
test_set.reset_index(inplace = True)

In [17]:
print(training_set.index)

RangeIndex(start=0, stop=4458, step=1)


Now the dictionary `word_count_per_sms` will be created. Each key will be one word from the list of unique words. The value of each key will be a list with the same length as the number of entries in the `training_set`. Each value in the list will be the number the word used as key appears in a single SMS message.

In [18]:
word_count_per_sms = {}
for word in vocabulary_lst:
    word_count_per_sms[word] = [0]*training_set.shape[0]
    
for index, sms in enumerate(training_set.SMS):
    for word in sms:
        word_count_per_sms[word][index] += 1

In [19]:
word_count = pd.DataFrame(word_count_per_sms)

Concacenating the labels with the new dataframe:

In [20]:
# training_set.reset_index(inplace = True)
tsdf = pd.concat([training_set, word_count], axis = 1)
tsdf = tsdf.drop('index', axis = 1)

print(training_set.shape, word_count.shape,tsdf.shape)
print(training_set.Label.value_counts(dropna = False))
print(training_set.index)
print(tsdf.head())


(4458, 3) (4458, 7783) (4458, 7784)
ham     3858
spam     600
Name: Label, dtype: int64
RangeIndex(start=0, stop=4458, step=1)
  Label                                                SMS  yep  by  the  \
0   ham                  [yep, by, the, pretty, sculpture]    1   1    1   
1   ham  [yes, princess, are, you, going, to, make, me,...    0   0    0   
2   ham                    [welp, apparently, he, retired]    0   0    0   
3   ham                                           [havent]    0   0    0   
4   ham  [i, forgot, 2, ask, ü, all, smth, there, s, a,...    0   0    0   

   pretty  sculpture  yes  princess  are  ...  beauty  hides  secrets  n8  \
0       1          1    0         0    0  ...       0      0        0   0   
1       0          0    1         1    1  ...       0      0        0   0   
2       0          0    0         0    0  ...       0      0        0   0   
3       0          0    0         0    0  ...       0      0        0   0   
4       0          0    0      

In [21]:
print(tsdf.Label.value_counts(dropna = False))

ham     3858
spam     600
Name: Label, dtype: int64


## Spam filter construction

The process of cleaning has been completed. Now the spam filter can be constructed.

The formula to gauge the probability that something is spam or ham is:

$$P(Spam|w_1,w_2,...,w_n) \propto P(Spam) \cdot \prod_{i = 1}^{n} P(w_{i}|Spam)$$

$$P(Ham|w_1,w_2,...,w_n) \propto P(Ham) \cdot \prod_{i = 1}^{n} P(w_{i}|Ham)$$

Where:

* $P(Spam|w_1,w_2,...,w_n)$ is the probability of spam given the words $w_i$ (Meaning the probability of a message being spam given the word $w_i$ is in the message)
* $P(Spam)$ is the probability that any word is spam
* $P(w_i|Spam)$ is the probability of $w_i$ given spam


And where the Probability for each word given it is spam or ham is:

$$P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}} $$

$$P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}} $$

Here:

* $\alpha$ is the smoothing operator. In the calculation Laplace smoothing will be used. In this case $\alpha = 1$.
* $N_{Vocabulary}$ is the total number of unique words our dataste
* $N_{Spam} (or N_{Ham})$ is the total number (not unique) of words in messages categorized as spam (ham)

The first step will be to calculate the constants that will be needed in the later calculation:

$N_{Spam}$ and $N_{Ham}$

In [26]:
n_spam = tsdf.loc[tsdf.Label == 'spam'].sum(axis = 1).sum()
n_ham = tsdf.loc[tsdf.Label == 'ham'].sum(axis = 1).sum()

In [27]:
print(n_spam)
print(n_ham)

15188
57237


Next: $P(Spam)$ & $P(Ham)$

In [28]:
spham_counts = tsdf.Label.value_counts(normalize = True)
p_spam = spham_counts['spam']
p_ham = spham_counts['ham']
print(p_spam)
print(p_ham)

0.13458950201884254
0.8654104979811574


$N_{Vocabulary}$

In [29]:
n_vocabulary = tsdf.shape[1] - 2
print(n_vocabulary)

7782


At this point it's a good idea to make another dataframe. It will have one column for spam and one for ham. The indices will be individual words and the values will be the number of times each word appears in each of the categories. This will cut down on computational expense, because when using $N_{W_i}$ the value only needs to be looked up, not counted each time.

In [31]:
words_occur = pd.DataFrame()
words_occur['spam'] = tsdf.loc[tsdf.Label == 'spam'].iloc[:,2:].sum(axis = 0)
words_occur['ham'] = tsdf.loc[tsdf.Label == 'ham'].iloc[:,2:].sum(axis = 0)

In [32]:
print(words_occur)

           spam  ham
yep           0    9
by           34  110
the         157  920
pretty        0   12
sculpture     0    1
...         ...  ...
related       0    1
trade         0    1
arul          0    1
bx526         1    0
wherre        0    1

[7782 rows x 2 columns]


To eliminate one further step in the calculation, instead of using the number of times a word occurs in each group, it will now be replaced by the probabilities of it appearing given spam and ham: $P(w_i|spam)$ and $P(w_i|ham)$

In [35]:
alpha = 1
words_probs = words_occur.copy()
words_probs.spam = (words_probs.spam + alpha) / (n_spam + (alpha * n_vocabulary))
words_probs.ham = (words_probs.ham + alpha)/ (n_ham + (alpha * n_vocabulary))

In [36]:
print(words_probs)

               spam       ham
yep        0.000044  0.000154
by         0.001524  0.001707
the        0.006879  0.014165
pretty     0.000044  0.000200
sculpture  0.000044  0.000031
...             ...       ...
related    0.000044  0.000031
trade      0.000044  0.000031
arul       0.000044  0.000031
bx526      0.000087  0.000015
wherre     0.000044  0.000031

[7782 rows x 2 columns]


In [60]:
print(test_set.head())

   index Label                                                SMS
0   2131   ham  [later, i, guess, i, needa, do, mcat, study, too]
1   3418   ham      [but, i, haf, enuff, space, got, like, 4, mb]
2   3424  spam  [had, your, mobile, 10, mths, update, to, late...
3   1538   ham  [all, sounds, good, fingers, makes, it, diffic...
4   5393   ham  [all, done, all, handed, in, don, t, know, if,...


In [39]:
def predictulator(row):
    '''Predict if a message is either ham or spam
    Args:
        row (int): row of the message to predict
    Returns:
        str: predicted category of the message
    '''
    spam_prob = p_spam
    ham_prob = p_ham
    for item in row:
        if item in words_probs.index:
            spam_prob *= words_probs.loc[item, 'spam']
            ham_prob *= words_probs.loc[item,'ham']
        else:
            spam_prob *= (alpha / (alpha * n_vocabulary))
            ham_prob *= (alpha / (alpha * n_vocabulary))
    if spam_prob > ham_prob:
        return 'spam'
    elif ham_prob > spam_prob:
        return 'ham'
    else:
        return'equal'

test_set['predict'] = test_set.SMS.apply(predictulator)  

In [40]:
print((test_set.Label == test_set.predict).value_counts(normalize = True)* 100)

True     98.743268
False     1.256732
dtype: float64


# Results

When using the accuracy metric, the spam filter was 98.7% accurate. This is a very good result. Unfortunately this metric does not take into account, that the large majority of messages are of the category ham. This could be looked at in more detail in further analysis.

# Summary

In this project, a spam filter was constructed for use on SMS messages, using a dataset of about 5000 messages. The spam filter was constructed using the naive Bayes algorithm. This algorithm is very good at categorizing messages. In total the spam filter scored 98.7% accuracy when categorizing the test set.